In [1]:
%load_ext autoreload
%autoreload 2

## Test Model

In [3]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="XiaomiMiMo/MiMo-VL-7B-RL",
    device="cuda",
    torch_dtype=torch.bfloat16,
)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=200)
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.21 GiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 21.31 GiB is allocated by PyTorch, and 543.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
from transformers import AutoProcessor
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

MODEL_PATH = "XiaomiMiMo/MiMo-VL-7B-RL"
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "url": "https://upload.wikimedia.org/wikipedia/commons/f/fa/Grayscale_8bits_palette_sample_image.png"
            },
            {
                "type": "text",
                "text": "describe this image"
            }
        ],
    }
]
processor = AutoProcessor.from_pretrained(MODEL_PATH, use_fast=True)
rl_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
).to(rl_model.device)
generated_ids = rl_model.generate(**inputs, max_new_tokens=8192)
output_text = processor.decode(generated_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=False)
print(output_text)


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

chat_template.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


KeyboardInterrupt: 

In [5]:
from transformers import AutoProcessor, Glm4vForConditionalGeneration
import torch

MODEL_PATH = "THUDM/GLM-4.1V-9B-Thinking"
device = "cuda"           # o "cuda:0"
dtype  = torch.bfloat16   # ocupa la mitad

processor = AutoProcessor.from_pretrained(MODEL_PATH, use_fast=True)
model = Glm4vForConditionalGeneration.from_pretrained(MODEL_PATH, torch_dtype=dtype, device_map="auto")
model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


Glm4vForConditionalGeneration(
  (model): Glm4vModel(
    (visual): Glm4vVisionModel(
      (embeddings): Glm4vVisionEmbeddings(
        (position_embedding): Embedding(576, 1536)
      )
      (patch_embed): Glm4vVisionPatchEmbed(
        (proj): Conv3d(3, 1536, kernel_size=(2, 14, 14), stride=(2, 14, 14))
      )
      (rotary_pos_emb): Glm4vVisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-23): 24 x Glm4vVisionBlock(
          (norm1): Glm4vRMSNorm((1536,), eps=1e-05)
          (norm2): Glm4vRMSNorm((1536,), eps=1e-05)
          (attn): Glm4vVisionAttention(
            (qkv): Linear(in_features=1536, out_features=4608, bias=False)
            (proj): Linear(in_features=1536, out_features=1536, bias=False)
          )
          (mlp): Glm4VisionMlp(
            (gate_proj): Linear(in_features=1536, out_features=4096, bias=False)
            (up_proj): Linear(in_features=1536, out_features=4096, bias=False)
            (down_proj): Linear(in_features=4096, out_features=1

In [8]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "url": "https://upload.wikimedia.org/wikipedia/commons/f/fa/Grayscale_8bits_palette_sample_image.png"
            },
            {
                "type": "text",
                "text": "describe this image"
            }
        ],
    }
]

clean = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_dict=True,
)

print("Messages with schema:")
print(clean)

inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device)

generated_ids = model.generate(**inputs, max_new_tokens=8192)
output_text = processor.decode(generated_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=False)
print(output_text)

Messages with schema:
[gMASK]<sop><|user|>
<|begin_of_image|><|image|><|end_of_image|>describe this image<|assistant|>



TypeError: group_images_by_shape() missing 1 required positional argument: 'disable_grouping'

In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient
mcp_url = "http://localhost:8001/mcp"

client = MultiServerMCPClient(
    {
        "balatro": {
            "transport": "streamable_http",
            "url": mcp_url,
        },
    }
)

tools = await client.get_tools()
tools

[StructuredTool(name='press_buttons', description="Press a sequence of buttons to control the Balatro game. These buttons are from Xbox controller: A, B, X, Y, LEFT, RIGHT, UP, DOWN, START, SELECT, RB, RT, LB, LT. Example: 'A B LEFT RB'", args_schema={'properties': {'sequence': {'title': 'Sequence', 'type': 'string'}}, 'required': ['sequence'], 'title': 'press_buttonsArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x768d59732700>),
 StructuredTool(name='get_screen', description='Get a screenshot of the current state of the Balatro game.', args_schema={'properties': {}, 'title': 'get_screenArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x768d59732200>)]

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import MessageGraph

def glm4v_call(messages, max_tokens=256):
    """messages = [SystemMessage | HumanMessage | AIMessage …]"""
    # 2.1 convierte a la lista dict{role,content} que espera el template
    m = []
    for msg in messages:
        # content multimodal → lista; texto → str puro
        m.append({"role": msg.type, "content": msg.content})

    # 2.2 tokeniza con el chat_template
    inputs = processor.apply_chat_template(
        m, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to(device)

    # 2.3 genera y decodifica
    out = model.generate(**inputs, max_new_tokens=max_tokens)
    reply = processor.decode(out[0][inputs["input_ids"].shape[1]:],
                             skip_special_tokens=False)
    return reply

# --- nodo LLM para el grafo ---
def llm_node(state):
    state["assistant_text"] = glm4v_call(state["prompt"])
    return state


In [ ]:
ACTION_RE = re.compile(r"Action:\s*(\{.*?\})", re.S)

def parse_route(state):
    txt = state["assistant_text"]
    m   = ACTION_RE.search(txt)
    if not m:
        return END, {"final_answer": txt}
    state["action"] = json.loads(m.group(1))
    return "call_tool", state

def call_tool(state):
    act = state["action"]
    name = act["action_type"]
    obs  = TOOLS[name].invoke(act | {})     # disparas la tool
    state["observation"] = obs
    return "bridge", state

def bridge(state):
    # empaqueta observación → mensaje user multimodal
    obs = state["observation"]
    user_msg = {"role": "user", "content": [obs] if isinstance(obs, dict) else obs}
    state["prompt"].extend([
        {"role":"assistant", "content": state["assistant_text"]},
        user_msg
    ])
    return "llm", state


In [ ]:
g = MessageGraph()
g.add_node("llm",       llm_node)
g.add_node("parse",     parse_route)
g.add_node("call_tool", call_tool)
g.add_node("bridge",    bridge)

g.add_edge("llm",   "parse")
g.add_edge("parse", "call_tool")
g.add_edge("call_tool", "bridge")
g.add_edge("bridge", "llm")

agent = g.compile()


## FULL

In [3]:
import re, json, asyncio
from langgraph.graph import MessageGraph, END
from langchain_mcp_adapters.client import MultiServerMCPClient

# ------------------------------------------------------------------
# 1.  Carga de herramientas MCP  ➜  lista de StructuredTool
# ------------------------------------------------------------------
mcp_url = "http://localhost:8001/mcp"
client  = MultiServerMCPClient({"balatro": {"transport": "streamable_http",
                                            "url": mcp_url}})
tools   = asyncio.run(client.get_tools())   # press_buttons / get_screen

# ------------------------------------------------------------------
# 2.  Expresiones y utilidades
# ------------------------------------------------------------------
ACTION_RE = re.compile(r'Action:\s*(\{.*?\})', re.S)   # captura JSON de la línea

# ------------------------------------------------------------------
# 3.  Nodo LLM : llama al modelo local GLM-4.1 V
# ------------------------------------------------------------------
def llm_node(state):
    reply = glm4v_call(state["prompt"])                # función wrapper → texto
    state["assistant"] = reply
    return state

# ------------------------------------------------------------------
# 4.  Nodo parse : decide si hay una acción
# ------------------------------------------------------------------
def parse_route(state):
    txt = state["assistant"]
    m   = ACTION_RE.search(txt)

    if not m:
        # 👉 no hay Action → flujo termina
        state["final"] = txt
        return END, state

    # 👉 se encontró Action → guarda y deriva a call_tool
    state["action"] = json.loads(m.group(1))
    return "call_tool", state

# ------------------------------------------------------------------
# 5.  Nodo call_tool : ejecuta la StructuredTool correspondiente
# ------------------------------------------------------------------
def call_tool(state):
    act   = state["action"]
    tname = act["action_type"]
    tool  = {t.name: t for t in tools}.get(tname)

    if tool is None:
        state["observation"] = f"⚠️ tool '{tname}' desconocida"
        return "bridge", state

    obs = tool.invoke(act | {})          # ejecuta (puede ser async → await)
    state["observation"] = obs           # imagen, texto, etc.
    return "bridge", state

# ------------------------------------------------------------------
# 6.  Nodo bridge : añade la observación y prepara el próximo prompt
# ------------------------------------------------------------------
def bridge(state):
    obs  = state["observation"]
    user_obs = {"role": "user",
                "content": [obs] if isinstance(obs, dict) else obs}

    # extiende el prompt para la próxima vuelta
    state["prompt"].extend([
        {"role": "assistant", "content"["assistant"]},
        user_obs
    ])
    return "llm", state                  # vuelve al nodo llm

# ------------------------------------------------------------------
# 7.  Grafo LangGraph
# ------------------------------------------------------------------
g = MessageGraph()
g.add_node("llm",       llm_node)
g.add_node("parse",     parse_route)
g.add_node("call_tool", call_tool)
g.add_node("bridge",    bridge)

g.add_edge("llm",       "parse")
g.add_edge("parse",     "call_tool")     # solo cuando hay Action
g.add_edge("call_tool", "bridge")
g.add_edge("bridge",    "llm")           # bucle
# parse → END cuando no hay Action

agent = g.compile(tools=tools)           # ⚙️   <<<<<


ImportError: cannot import name 'State' from 'langgraph.graph' (/home/javi/miniconda3/envs/jokernet/lib/python3.11/site-packages/langgraph/graph/__init__.py)